## Ad Image Insertion in Video with Occlusion Handling 

In [31]:
import cv2
import numpy as np


#Inserts a resized image into the top-left corner of a video and creates a new video file
def insert_image_into_video(video_path, image_path, output_path):
    # Read video and image
    video = cv2.VideoCapture(video_path)
    
    # Check if the image is successfully loaded
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found or could not be loaded: {image_path}")

    # Get video properties
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)

    # Define VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use appropriate codec
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Specify the position for the top left corner
    x_offset, y_offset = 100, 450

    while True:
        ret, frame = video.read()

        if not ret:
            break

        # Copy the frame to avoid modifying the original
        frame_with_logo = frame.copy()

        # Resize the image to fit in the top left corner
        logo_resized = cv2.resize(image, (int(width/5), int(height/5)))

        # Overlay the resized logo on the frame
        frame_with_logo[y_offset:y_offset+logo_resized.shape[0], x_offset:x_offset+logo_resized.shape[1]] = logo_resized

        out.write(np.uint8(frame_with_logo))

        cv2.imshow('Video with Logo Insertion', np.uint8(frame_with_logo))
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    video.release()
    out.release()
    cv2.destroyAllWindows()

#File_Path
video_path = r'C:\Users\param\Documents\Fun\Projects\Video_Occlusion_Handling\Input Video 2.mp4'
image_path = r'C:\Users\param\Documents\Fun\Projects\Video_Occlusion_Handling\Advertisement Image.jpg'
output_path = r'C:\Users\param\Documents\Fun\Projects\Video_Occlusion_Handling\ex_video\output_video_with_logo.mp4'

#Error_Handling
try:
    insert_image_into_video(video_path, image_path, output_path)
except Exception as e:
    print(f"An error occurred: {e}")


## Ad Image Insertion in Video with Occlusion Handling using MediaPipe Hands model

In [32]:
import cv2
import numpy as np
import mediapipe as mp
import os
import wget

# Set TensorFlow to ignore warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

def detect_hand(video_path, image_path, output_path, base_hand_level=0.3, image_scale=0.2):
    # Initialize MediaPipe Hands
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands()

    # Read image
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

    # Ensure the image has an alpha channel
    if image.shape[2] == 3:  # If the image is RGB, add an alpha channel
        image = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)

    # Set initial position for the image in the top-left corner
    top_left_y = 450
    top_left_x = 100

    # Get video properties
    video = cv2.VideoCapture(video_path)
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)

    # Define VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use appropriate codec
    out_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while True:
        ret, frame = video.read()

        if not ret:
            break

        # Find the hand landmarks using MediaPipe Hands
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            for landmarks in results.multi_hand_landmarks:
                hand_landmarks = np.array([(lmk.x * width, lmk.y * height) for lmk in landmarks.landmark])

                hand_tip_x, hand_tip_y = hand_landmarks[8]
                hand_tip_x, hand_tip_y = int(hand_tip_x), int(hand_tip_y)

                # Check if the hand is above a certain level
                if hand_tip_y < base_hand_level * height:
                    # Move the image when the hand is above the specified level
                    top_left_y = max(0, hand_tip_y - int(image_scale * height))

                # Resize the image to fit the hand region
                resized_image = cv2.resize(image, (int(image_scale * width), int(image_scale * height)))

                # Overlay the image in the top-left corner
                alpha_channel = cv2.resize(resized_image[:, :, 3], (resized_image.shape[1], resized_image.shape[0])) / 255.0
                frame[top_left_y:top_left_y+resized_image.shape[0], top_left_x:top_left_x+resized_image.shape[1]] = (
                    (1 - alpha_channel)[:, :, np.newaxis] * frame[top_left_y:top_left_y+resized_image.shape[0], top_left_x:top_left_x+resized_image.shape[1]]
                    + alpha_channel[:, :, np.newaxis] * resized_image[:, :, :3]
                )

        out_writer.write(frame)

        cv2.imshow('Video with Image Insertion', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    video.release()
    out_writer.release()
    cv2.destroyAllWindows()


video_path = r'C:\Users\param\Documents\Fun\Projects\Video_Occlusion_Handling\Input Video 2.mp4'
image_path = r'C:\Users\param\Documents\Fun\Projects\Video_Occlusion_Handling\Advertisement Image.jpg'
output_path = r'C:\Users\param\Documents\Fun\Projects\Video_Occlusion_Handling\ex_video\output_video_with_logo.mp4'

detect_hand(video_path, image_path, output_path)
